In [ ]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [ ]:
import re
import nltk
from textblob import Word
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import csv
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df = pd.read_csv('/content/Covid-19 Twitter Dataset (Aug-Sep 2020).csv')

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
tweets_column = df['clean_tweet']

In [ ]:
compound = df['compound']
sentiment = df['sentiment']

In [ ]:
print(tweets_column)

0                 year old ex vice presid moodi awori land inter counti covid19 committe role
1                      break depart health report peopl caught covid19 push nation case count
2                                    help fan request help arrang bed posit father view tweet
3                                                              lend club loan origin hey bank
4                     curiou case unit nation ngo appreci kingdom mamata banerje covid19 work
                                                 ...                                         
120504                      exclus astrazeneca covid19 vaccin trial may resum soon week sourc
120505                         worker countri sign petit reinstat worker mass fire request co
120506    cultur china brillianc huax celebr th anniversari canada china diplomat relat fight
120507                               trump call cnn bastard cover covid19 hear keilar respons
120508                           trump call cnn bastard cove

In [ ]:
print(compound)
print(sentiment)

0         0.0000
1         0.0000
2         0.7717
3         0.0000
4         0.0000
           ...  
120504    0.0000
120505   -0.3400
120506   -0.3818
120507   -0.5423
120508   -0.5423
Name: compound, Length: 120509, dtype: float64
0         neu
1         neu
2         pos
3         neu
4         neu
         ... 
120504    neu
120505    neg
120506    neg
120507    neg
120508    neg
Name: sentiment, Length: 120509, dtype: object


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
import re

def processRow(row):
    row = row.lower()
    row = re.sub(r'(\\u[0-9A-Fa-f]+)', "", row)
    row = re.sub(r'[^\x00-\x7f]', "", row)
    row = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', 'URL', row)
    row = re.sub('@[^\s]+', 'AT_USER', row)
    row = re.sub(r'[\s]+', ' ', row)
    row = re.sub(r'[\n]+', ' ', row)
    row = re.sub(r'[^\w]', ' ', row)
    row = re.sub(r'#([^\s]+)', r'\1', row)
    row = row.replace(':)', "")
    row = row.replace(':(', "")
    row = "".join([i for i in row if not i.isdigit()])
    row = re.sub(r"(\!)\1+", ' ', row)
    row = re.sub(r"(\?)\1+", ' ', row)
    row = re.sub(r"(\.)\1+", ' ', row)

    lemmatizer = WordNetLemmatizer()
    tokenizer = TweetTokenizer()

    words = tokenizer.tokenize(row)
    stop_words = set(stopwords.words('english'))
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    filtered_sentence = ' '.join(filtered_words)
    return filtered_sentence


In [ ]:
data = tweets_column.apply(lambda x: processRow(str(x)) if isinstance(x, str) else str(x))

In [ ]:
datas=data.apply(processRow)

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

tweets=datas

lemmatized_tweets = []

for tweet in tweets:
    words = word_tokenize(tweet)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized_tweet = ' '.join(lemmatized_words)
    lemmatized_tweets.append(lemmatized_tweet)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
sentences = lemmatized_tweets

stemmer = PorterStemmer()

stemmed_sentences = []
for sentence in sentences:
    words = word_tokenize(sentence)
    stemmed_words = [stemmer.stem(word) for word in words]
    stemmed_sentence = ' '.join(stemmed_words)
    stemmed_sentences.append(stemmed_sentence)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

sentences = stemmed_sentences

all_words = []

for sentence in sentences:
    words = word_tokenize(sentence)
    all_words.extend(words)

print("Extracted words:", all_words)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
postive={}
negative={}
neutral={}
for i,j in zip(sentiment,lemmatized_tweets):
  if i =='pos':
    postive[j]='positive'
  elif i=='neg':
    negative[j]='negative'
  else:
    neutral[j]='neutral'
print(len(postive))
print(len(negative))

35021
31151


In [ ]:
tweet_data=[]
label=[]
for i,j in zip(sentiment,lemmatized_tweets):
  if i =='pos':
    tweet_data.append(j);
    label.append('positive')
  elif i=='neg':
    tweet_data.append(j);
    label.append('negative')
print(len(tweet_data))
print(len(label))
print(tweet_data[1])
print(label[1])

69515
69515
jtbc drama privat life star snsd seohyun go kyungpyo stop film due covid concern wi
negative


In [ ]:
!pip install fasttext

Multinomial Naive Bayes

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
combined_train_texts, combined_test_texts, combined_train_labels, combined_test_labels = train_test_split(
    tweet_data,label, test_size=0.3, random_state=42
)


vectorizer = TfidfVectorizer()

combined_train_features = vectorizer.fit_transform(combined_train_texts)
combined_test_features = vectorizer.transform(combined_test_texts)
nb_combined_model = MultinomialNB()
nb_combined_model.fit(combined_train_features, combined_train_labels)

combined_test_predictions = nb_combined_model.predict(combined_test_features)


combined_accuracy = accuracy_score(combined_test_labels, combined_test_predictions)
print(f"Combined Accuracy: {combined_accuracy:.2f}")

Combined Accuracy: 0.88


Linear SVC

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score


positive_data =postive
negative_data = negative

combined_data = {**positive_data, **negative_data}


combined_texts = list(combined_data.keys())
combined_labels = list(combined_data.values())


combined_train_texts, combined_test_texts, combined_train_labels, combined_test_labels = train_test_split(
    tweet_data, label, test_size=0.3, random_state=42
)


vectorizer = TfidfVectorizer()


combined_train_features = vectorizer.fit_transform(combined_train_texts)
combined_test_features = vectorizer.transform(combined_test_texts)

svc_model = LinearSVC()
svc_model.fit(combined_train_features, combined_train_labels)

combined_test_predictions = svc_model.predict(combined_test_features)


combined_accuracy = accuracy_score(combined_test_labels, combined_test_predictions)
print(f"Combined Accuracy using LinearSVC: {combined_accuracy:.2f}")


Combined Accuracy using LinearSVC: 0.97


Random Forest Classifier

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
train_features, test_features, train_labels, test_labels = train_test_split(
    tweet_data, label, test_size=0.3, random_state=42
)


vectorizer = TfidfVectorizer()

train_features_tfidf = vectorizer.fit_transform(train_features)
test_features_tfidf = vectorizer.transform(test_features)

rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(train_features_tfidf, train_labels)


test_predictions = rf_model.predict(test_features_tfidf)


accuracy = accuracy_score(test_labels, test_predictions)

print(f"Accuracy using RandomForestClassifier: {accuracy:.2f}")


Accuracy using RandomForestClassifier: 0.94


Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

vectorizer = TfidfVectorizer()
combined_features = vectorizer.fit_transform(tweet_data)

train_features, test_features, train_labels, test_labels = train_test_split(
    combined_features, label, test_size=0.3, random_state=42
)


logistic_combined_model = LogisticRegression()
logistic_combined_model.fit(train_features, train_labels)


test_predictions = logistic_combined_model.predict(test_features)


accuracy = accuracy_score(test_labels, test_predictions)

print(f"Accuracy using Logistic Regression: {accuracy:.2f}")


Accuracy using Logistic Regression: 0.96
